In [2]:
email_conversation = """From: 테디 (teddy@teddynote.com)
To: 이은채 대리님 (eunchae@teddyinternational.me)
Subject: RAG 솔루션 시연 관련 미팅 제안

안녕하세요, 이은채 대리님,

저는 테디노트의 테디입니다. 최근 귀사에서 AI를 활용한 혁신적인 솔루션을 모색 중이라는 소식을 들었습니다. 테디노트는 AI 및 RAG 솔루션 분야에서 다양한 경험과 노하우를 가진 기업으로, 귀사의 요구에 맞는 최적의 솔루션을 제공할 수 있다고 자부합니다.

저희 테디노트의 RAG 솔루션은 귀사의 데이터 활용을 극대화하고, 실시간으로 정확한 정보 제공을 통해 비즈니스 의사결정을 지원하는 데 탁월한 성능을 보입니다. 이 솔루션은 특히 다양한 산업에서의 성공적인 적용 사례를 통해 그 효과를 입증하였습니다.

귀사와의 협력 가능성을 논의하고, 저희 RAG 솔루션의 구체적인 기능과 적용 방안을 시연하기 위해 미팅을 제안드립니다. 다음 주 목요일(7월 18일) 오전 10시에 귀사 사무실에서 만나 뵐 수 있을까요?

미팅 시간을 조율하기 어려우시다면, 편하신 다른 일정을 알려주시면 감사하겠습니다. 이은채 대리님과의 소중한 만남을 통해 상호 발전적인 논의가 이루어지길 기대합니다.

감사합니다.

테디
테디노트 AI 솔루션팀"""

In [5]:
from pydantic import BaseModel, Field


# 이메일 본문으로부터 주요 엔티티 추출
# 필드안의 description이 중요!
# 꼭 필요한 정보를 정의해야함


# OutputParser에 들어가는 것
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    company: str = Field(description="메일을 보낸 사람의 회사 정보")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")

In [6]:
# LCEL 구조 확립
# chain = prompt | llm | output_parser

### LLM

In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model_name="gpt-4o",
    temperature=0,
)

### Output Parser

In [8]:
from langchain_core.output_parsers import PydanticOutputParser

output_parser = PydanticOutputParser(pydantic_object=EmailSummary)

### Prompt

- 프롬프트는 영어로 작성하는 것을 추천

In [9]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

#QUESTION:
다음의 이메일 내용 중에서 주요 내용을 추출해 주세요.

#EMAIL CONVERSATION:
{email_conversation}

#FORMAT:
{format}
"""
)

# format 에 PydanticOutputParser의 부분 포맷팅(partial) 추가
prompt = prompt.partial(format=output_parser.get_format_instructions())

### 체인 생성

In [10]:
chain = prompt | llm | output_parser

### 체인 실행

In [11]:
answer = chain.invoke({"email_conversation": email_conversation})

In [12]:
print(answer)

person='테디' company='테디노트' email='teddy@teddynote.com' subject='RAG 솔루션 시연 관련 미팅 제안' summary='테디노트의 테디가 이은채 대리님에게 AI 및 RAG 솔루션 시연을 위한 미팅을 제안하며, 솔루션의 기능과 적용 방안을 설명하고 협력 가능성을 논의하고자 한다.' date='7월 18일 오전 10시'


## 검색: SERP API

- 참고: https://serpapi.com/integrations/python
- 무료 버전은 최대 100건 한정

In [30]:
from dotenv import load_dotenv

load_dotenv()

True

In [31]:
from langchain_community.utilities import SerpAPIWrapper

# gl: Country
# hl: UI Language
# num: 검색 결과 몇 건을 가지고 올 것인가?
params = {"engine": "google", "gl": "kr", "hl": "ko", "num": "3"}

search = SerpAPIWrapper(params=params)

In [32]:
# 테스트
search.run("테디노트 site:naver.com")

'[\'패스트캠퍼스에서 진행 중인 테디노트의 RAG 비법노트를 덮썩 물었다. ... RAG& 랭체인 개발자의 테디노트 선생님의 영상을 보고 책을 구매하게 된다.\', "... 테디노트 (teddylee777.github.io)운영(\'17년~현재) 전 - 삼성전자SCSA1기 - 삼성전자무선사업부삼성노트,테마,갤러리 앱개발 - C-LAB사내벤처독립 ..."]'

이메일에서 추출한 정보로 검색 진행

In [17]:
query = f"{answer.person} {answer.company} {answer.email}"
query

'테디 테디노트 teddy@teddynote.com'

In [20]:
search_result = eval(search.run(query))

In [23]:
type(search_result[0])

str

In [27]:
# 검색 결과
search_result_string = "\n".join(search_result)
print(search_result_string)

테디노트 X 패스트캠퍼스 "RAG 비법노트" · 환경 설정 (Mac) · 환경 설정 (Windows). LocalModels. GGUF · HuggingFace gguf 파일을 Ollama 로딩 · TeddyNote.
테디노트 데이터와 인공지능을 좋아하는 개발자 노트 · 검색. 토글 메뉴. 카테고리 · 태그 · 연도 · 강의 · 어바웃미 · Teddy. Creator & Data Lover. 팔로우. Pangyo, ...
데이터 분석, 머신러닝, 딥러닝, LLM 에 대한 내용을 다룹니다. 연구보다는 개발에 관심이 많습니다 ‍♂️ ...more ...more fastcampus.co.kr/data_online_teddyand 2 ...


In [28]:
answer # 필요한 정보가 담겨있는 변수

EmailSummary(person='테디', company='테디노트', email='teddy@teddynote.com', subject='RAG 솔루션 시연 관련 미팅 제안', summary='테디노트의 테디가 이은채 대리님에게 AI 및 RAG 솔루션 시연을 위한 미팅을 제안하며, 솔루션의 기능과 적용 방안을 설명하고 협력 가능성을 논의하고자 한다.', date='7월 18일 오전 10시')

- additional_information은 정보 검색을 통해 찾아낸 정보를 넣음

In [29]:
from langchain_core.prompts import PromptTemplate

report_prompt = PromptTemplate.from_template(
    """당신은 이메일의 주요 정보를 바탕으로 요약 정리해 주는 전문가 입니다.
당신의 임무는 다음의 이메일 정보를 바탕으로 보고서 형식의 요약을 작성하는 것입니다.
주어진 정보를 기반으로 양식(format)에 맞추어 요약을 작성해 주세요.

#Information:
- Sender: {sender}
- Additional Information about sender: {additional_information}
- Company: {company}
- Email: {email}
- Subject: {subject}
- Summary: {summary}
- Date: {date}

#Format(in markdown format):
🙇‍♂️ 보낸 사람:
- (보낸 사람의 이름, 회사 정보)

📧 이메일 주소:
- (보낸 사람의 이메일 주소)

😍 보낸 사람과 관련하여 검색된 추가 정보:
- (검색된 추가 정보)

✅ 주요 내용:
- (이메일 제목, 요약)

⏰ 일정:
- (미팅 날짜 및 시간)

#Answer:"""
)

In [33]:
from langchain_core.output_parsers import StrOutputParser

In [36]:
report_chain = report_prompt | ChatOpenAI(model="gpt-4o", temperature=0) | StrOutputParser()

In [37]:
report_response = report_chain.invoke({
    "sender": answer.person,
    "additional_information": search_result_string,
    "company": answer.company,
    "email": answer.email,
    "subject": answer.subject,
    "summary": answer.summary,
    "date": answer.date,
})

In [38]:
print(report_response)

```markdown
🙇‍♂️ 보낸 사람:
- 테디, 테디노트

📧 이메일 주소:
- teddy@teddynote.com

😍 보낸 사람과 관련하여 검색된 추가 정보:
- 테디노트 X 패스트캠퍼스 "RAG 비법노트" · 환경 설정 (Mac) · 환경 설정 (Windows). LocalModels. GGUF · HuggingFace gguf 파일을 Ollama 로딩 · TeddyNote.
- 테디노트는 데이터와 인공지능을 좋아하는 개발자 노트로, 데이터 분석, 머신러닝, 딥러닝, LLM에 대한 내용을 다루며, 연구보다는 개발에 관심이 많습니다.
- 테디는 Creator & Data Lover로 Pangyo에 위치하며, 다양한 강의와 프로젝트를 진행하고 있습니다.

✅ 주요 내용:
- 이메일 제목: RAG 솔루션 시연 관련 미팅 제안
- 요약: 테디노트의 테디가 이은채 대리님에게 AI 및 RAG 솔루션 시연을 위한 미팅을 제안하며, 솔루션의 기능과 적용 방안을 설명하고 협력 가능성을 논의하고자 합니다.

⏰ 일정:
- 7월 18일 오전 10시
```
